In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv('Smarket.csv', index_col=0, parse_dates=True)
df.describe()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
count,1250.000000,1250.000000,1250.000000,1250.000000,1250.00000,1250.000000,1250.000000
mean,0.003834,0.003919,0.001716,0.001636,0.00561,1.478305,0.003138
std,1.136299,1.136280,1.138703,1.138774,1.14755,0.360357,1.136334
min,-4.922000,-4.922000,-4.922000,-4.922000,-4.92200,0.356070,-4.922000
25%,-0.639500,-0.639500,-0.640000,-0.640000,-0.64000,1.257400,-0.639500
50%,0.039000,0.039000,0.038500,0.038500,0.03850,1.422950,0.038500
75%,0.596750,0.596750,0.596750,0.596750,0.59700,1.641675,0.596750
max,5.733000,5.733000,5.733000,5.733000,5.73300,3.152470,5.733000


In [3]:
df.head()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [4]:
import statsmodels.formula.api as smf
df['Direction'] = df['Direction'].map({'Up': 0, 'Down': 1})

In [5]:
formula = 'Direction ~ Lag1+Lag2+Lag3+Lag4+Lag5+Volume'

In [6]:
model = smf.glm(formula=formula, data=df, family=sm.families.Binomial())
result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1250
Model:                            GLM   Df Residuals:                     1243
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -863.79
Date:                Tue, 03 Apr 2018   Deviance:                       1727.6
Time:                        14:14:06   Pearson chi2:                 1.25e+03
No. Iterations:                     4                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1260      0.241      0.523      0.601      -0.346       0.598
Lag1           0.0731      0.050      1.457      0.1

In [7]:
predictions = result.predict()
print(predictions[0:10])


[0.49291587 0.51853212 0.51886117 0.48477764 0.48921884 0.49304354
 0.50734913 0.49077084 0.48238647 0.51116222]


In [8]:
print(np.column_stack((df.as_matrix(columns=["Direction"]).flatten(), result.model.endog)))

[[0. 0.]
 [0. 0.]
 [1. 1.]
 ...
 [0. 0.]
 [1. 1.]
 [1. 1.]]


In [9]:
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]

In [10]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(df["Direction"], predictions_nominal))

[[507 141]
 [457 145]]


In [11]:
print(classification_report(df["Direction"], predictions_nominal, digits=3))

             precision    recall  f1-score   support

          0      0.526     0.782     0.629       648
          1      0.507     0.241     0.327       602

avg / total      0.517     0.522     0.483      1250



Result is quite poor - a little bit better than random guessing - (training error rate = 1-52,2% = 47,8%). But prediction is still made on the same data as training. Let's slice it into train and test.

In [12]:
X_train = df.loc[:'2004'][:]
y_train = df[:'2004']['Direction']


X_test = df['2005':][['Lag1','Lag2','Lag3','Lag4','Lag5','Volume','Today']]
y_test = df['2005':]['Direction']


In [13]:
formula = 'Direction ~ Lag1+Lag2+Lag3+Lag4+Lag5+Volume'
model = smf.glm(formula=formula, data=X_train, family=sm.families.Binomial())
result = model.fit()

In [14]:
predictions = result.predict(X_test)
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
print(classification_report(y_test, predictions_nominal, digits=3))

             precision    recall  f1-score   support

          0      0.564     0.312     0.402       141
          1      0.443     0.694     0.540       111

avg / total      0.511     0.480     0.463       252



test error rate (1 - recall) is 52% - result is even worst.

Time to play aorund a little bit with some classification models.

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


In [16]:
X_train = df.loc[:'2004'][['Lag1','Lag2','Lag3','Lag4','Lag5','Volume','Today']]
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [17]:
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test.values)
        predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
        print(name)
        print(classification_report(y_test, predictions_nominal, digits=3))
        score = clf.score(X_test, y_test)
        print(name +': =' + str(score))



Nearest Neighbors
             precision    recall  f1-score   support

          0      0.891     0.865     0.878       141
          1      0.835     0.865     0.850       111

avg / total      0.866     0.865     0.865       252

Nearest Neighbors: =0.8650793650793651
Linear SVM
             precision    recall  f1-score   support

          0      0.986     0.993     0.989       141
          1      0.991     0.982     0.986       111

avg / total      0.988     0.988     0.988       252

Linear SVM: =0.9880952380952381
RBF SVM
             precision    recall  f1-score   support

          0      0.931     0.865     0.897       141
          1      0.843     0.919     0.879       111

avg / total      0.892     0.889     0.889       252

RBF SVM: =0.8888888888888888
Gaussian Process
             precision    recall  f1-score   support

          0      1.000     0.993     0.996       141
          1      0.991     1.000     0.996       111

avg / total      0.996     0.996     0.9

'\n        # Plot the decision boundary. For that, we will assign a color to each\n        # point in the mesh [x_min, x_max]x[y_min, y_max].\n        if hasattr(clf, "decision_function"):\n            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])\n        else:\n            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]\n\n        # Put the result into a color plot\n        Z = Z.reshape(xx.shape)\n        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)\n\n        # Plot also the training points\n        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,\n                   edgecolors=\'k\')\n        # and testing points\n        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,\n                   edgecolors=\'k\', alpha=0.6)\n\n        ax.set_xlim(xx.min(), xx.max())\n        ax.set_ylim(yy.min(), yy.max())\n        ax.set_xticks(())\n        ax.set_yticks(())\n        if ds_cnt == 0:\n            ax.set_title(name)\n        ax.text(x